In [20]:
import sys
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [78]:
filepaths=!ls *.csv
filepaths=[i for i in filepaths if '@20' in i]
filepaths

['ADMM@20.csv',
 'CDAE@20.csv',
 'EASE@20.csv',
 'FFM@20.csv',
 'GRU4RecF@20.csv',
 'ItemKNN@20.csv',
 'LGCN@20.csv',
 'MultiDAE@20.csv',
 'NeuMF@20.csv',
 'Pop@20.csv',
 'RecVAE@20.csv',
 'SLIMElastic@20.csv']

In [79]:
ratios=[1,0.2,1,0.5,0.1,0.4,0.1,0.6,0.6,0.1,1,0.3,0.6]

In [80]:
w_dict = dict(zip(filepaths,ratios))
w_dict

{'ADMM@20.csv': 1,
 'CDAE@20.csv': 0.2,
 'EASE@20.csv': 1,
 'FFM@20.csv': 0.5,
 'GRU4RecF@20.csv': 0.1,
 'ItemKNN@20.csv': 0.4,
 'LGCN@20.csv': 0.1,
 'MultiDAE@20.csv': 0.6,
 'NeuMF@20.csv': 0.6,
 'Pop@20.csv': 0.1,
 'RecVAE@20.csv': 1,
 'SLIMElastic@20.csv': 0.3}

In [81]:
dataframe_list = []

print('순위별 가중치값 입력(ex: 1 0.9 0.8 ...)')
rank_ratio = [1 for _ in range(10)] + [0.3 for _ in range(5)]
rank_len = len(rank_ratio)

print(f"앙상블 모델 개수: {len(filepaths)}")

for i in range(len(filepaths)):
    dataframe_list.append(pd.read_csv(filepaths[i]))

user_list = dataframe_list[0]['user'].unique()
dataframe_len = len(dataframe_list)

순위별 가중치값 입력(ex: 1 0.9 0.8 ...)
앙상블 모델 개수: 12


In [68]:
len(user_list)

31360

In [69]:
ratios = list(w_dict.values())
K=10
result = []
tbar = tqdm(user_list, desc='Ensemble')
for user in tbar:
    temp = defaultdict(float)
    for df_idx in range(dataframe_len):
        items = dataframe_list[df_idx][dataframe_list[df_idx]['user'] == user]['item'].values
        max_rank = min(len(items), rank_len)
        for rank_idx in range(max_rank):
            temp[items[rank_idx]] += rank_ratio[rank_idx] * ratios[df_idx]

    for key, _ in sorted(temp.items(), key=lambda x: x[1], reverse=True)[:K]:
        result.append((user, key))

Ensemble: 100%|██████████████████████████████████████████████| 31360/31360 [06:00<00:00, 87.05it/s]


In [82]:
filename="--".join([f"{m.split('@')[0]}_{r}" for m,r in zip(filepaths[:-1],ratios[:-1]) if "@" in m])
filename

'ADMM_1--CDAE_0.2--EASE_1--FFM_0.5--GRU4RecF_0.1--ItemKNN_0.4--LGCN_0.1--MultiDAE_0.6--NeuMF_0.6--Pop_0.1--RecVAE_1'

In [71]:
submission = pd.DataFrame(result, columns=['user', 'item'])
submission.to_csv(f'{filename}.csv', index=False)